In [1]:
#add mbes_sim to search path
import sys
sys.path.insert(0, "../../src/")

In [2]:
%matplotlib widget

from ipywidgets import *
import matplotlib.pyplot as plt

import numpy as np
import math
from copy import deepcopy
from tqdm.auto import tqdm
import pandas as pd

import mbes_sim.simulationfunctions as SIMFUN
import mbes_sim.functions.create_bubblesfunctions as bubbles

In [3]:
# simplify plotting
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 100
close_plots = True


def create_figure(name):
    if close_plots: plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name
    
    return fig

In [4]:
downfactor = 1 #speed up sumlation by reducing the resolution and number of beams and samples of the mbes
resfactor  = 1
voxsize    = 1

beam_pattern = SIMFUN.t_Window.Exponential

method_name = 'sv_int_lin'

In [5]:
setup = SIMFUN.SimulationSetup(
    addPlots = True,
    prefix = 'examples',
    downfactor = downfactor,
    blockAvg = False,
    resfactor  = resfactor,
    windowType           = beam_pattern,
    idealizedBeampattern = False,
    equiDist             = False,
    motionDataPath = '../../test_data/m143_l0154_motion.csv',
    
    surveyType           = SIMFUN.t_Survey.RealMotion,
    voxelsize            = voxsize / downfactor,
    voxelsizeZ           = voxsize / downfactor,
    surveyspeedKnots     = 3,
    swathdistance        = 0.8 / downfactor,
    layerDepths          = [],
    layerSizes           = [],
    bubbleType           = SIMFUN.t_Bubbles.BubbleStream,
    exagHPR              = 1,
    BaseDirectory        = 'GEOMAR_simresults',
    
    load_previous_simresults = False,    
    verbose = True)


  0%|          | 0/256 [00:00<?, ?it/s]

recalculated MBES
-108.25 108.25


In [6]:
sim_setup = deepcopy(setup)

sim_survey = sim_setup.get_survey()

sim_survey.setRandomOffsetX(sim_setup.SimSetup['voxelsize'])
sim_survey.setRandomOffsetY(sim_setup.SimSetup['voxelsize'])
sim_survey.setRandomOffsetZ(sim_setup.SimSetup['voxelsize'])
sim_survey.pitchs_degree+=0 # try adding pitch. It is interesting as it bends the bubble stream side lobes (but does not seem to have a negative effect on echo grid integration)
sim_survey.yaws_degree+=0

sim_setup.Simulation.setSurvey(sim_survey)

targets = bubbles.BubbleGenerator().generate_bubblestreams_within_cylindrical_section_along_path(
    start_x = -1.5,
    end_x = 1.5,
    min_range = 0,
    max_range = 130,
    min_beamsteeringangle = -90,
    max_beamsteeringangle = 90,
    min_y = 41,
    max_y = 43,
        zdist_list=[0.1])

if False:
    xbase=targets.x[int(len(targets)/2)]
    ybase=targets.y[int(len(targets)/2)]
    zbase=targets.z[int(len(targets)/2)]
    for i in range(len(targets)):
        targets.x[i] += (targets.z[i] - zbase)/1.1 - xbase
        targets.y[i] += (targets.z[i] - zbase)/1.12 - ybase

sim_setup.Simulation.setTargets(targets)

np.mean(targets.y)

np.float64(42.29803219146236)

In [8]:
SV,X,Y,Z = sim_setup.Simulation.simulate3DEchoesSamples(progress=True)
sv=SV.flatten()
arg = np.where(np.isfinite(sv))
sv=sv[arg]
x=X.flatten()[arg]
y=Y.flatten()[arg]
z=Z.flatten()[arg]

  0%|          | 0/271 [00:00<?, ?it/s]

124.76800000000003 107.4531368318111 -1.1217158686502797 1228 1228                            

In [12]:
data = pd.DataFrame()
data['x']=x
data['y']=y
data['depth']=z
data['altitude']=-z
data['sv']=sv
data['Sv']=10*np.log10(sv)

/tmp/ipykernel_65814/3029766010.py:7: RuntimeWarning: divide by zero encountered in log10
  data['Sv']=10*np.log10(sv)


In [14]:
data.to_csv('export.csv')

In [17]:
data.to_hdf('export.h5',key='data')